# Setup Libraries

In [1]:
import sys #append for path
import importlib
import os
import psycopg2
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta,date
import math
import calendar
from io import StringIO
from dotenv import load_dotenv
load_dotenv('osama.env')
import json
import requests
from metabasepy import Client, MetabaseTableParser
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import json
import requests
from io import StringIO
import pandas as pd
def ret_metabase(question_id, filters = {}):
    base_url = 'https://metabase.maxab.info:8444/api'
    base_headers = {'Content-Type': 'application/json'}
    try:
        s_response = requests.post(
            base_url + '/session',
            data=json.dumps({'username': os.environ["METABASE_USER_NAME"], 'password': os.environ["METABASE_PASSWORD"]}),
            headers=base_headers
        )
        s_response.raise_for_status()
        session_token = s_response.json()['id']
        base_headers['X-Metabase-Session'] = session_token
        params = []
        for name, value in filters.items():
            filter_type, filter_value = value
            if filter_type.lower() == 'date':
                if isinstance(filter_value, list):
                    params.append({'type': 'date/range', 'target': ['variable', ['template-tag', name]], 'value': filter_value})
                else:
                    params.append({'type': 'date/single', 'target': ['variable', ['template-tag', name]], 'value': filter_value})
            elif filter_type.lower() == 'category':
                params.append({'type': 'category', 'target': ['variable', ['template-tag', name]], 'value': filter_value})
            elif filter_type.lower() == 'text':
                params.append({'type': 'text', 'target': ['variable', ['template-tag', name]], 'value': filter_value})
            elif filter_type.lower() == 'number':
                params.append({'type': 'number', 'target': ['variable', ['template-tag', name]], 'value': filter_value})
            elif filter_type.lower() == 'field list':
                params.append({'type': 'id', 'target': ['dimension', ['template-tag', name]], 'value': filter_value})
                
        p_response = requests.post(
            base_url + '/card/' + str(question_id) + '/query/csv',
            params={'parameters': json.dumps(params)},
            headers=base_headers
        )
        p_response.raise_for_status()
        my_dict = p_response.content
        s = str(my_dict, 'utf-8')
        my_dict = StringIO(s)
        df = pd.read_csv(my_dict)
        return df
    except requests.exceptions.HTTPError as errh:
        return errh
    except requests.exceptions.ConnectionError as errc:
        return errc
    except requests.exceptions.Timeout as errt:
        return errt
    except requests.exceptions.RequestException as err:
        return err

In [3]:
# cli = Client(username=os.environ["METABASE_USER_NAME"], password=os.environ["METABASE_PASSWORD"], 
#          base_url='https://metabase.maxab.info:8444/api')
# cli.authenticate()

In [4]:
dirr1 = 'HH_'+str(date.today())+'_districts'
isdir = os.path.isdir(dirr1)
if(not isdir):
    os.mkdir(str(dirr1))
else:
    print("Directory available")

Directory available


In [5]:
now = datetime.now()
current_time_data = now.strftime("%d/%m/%Y %H:%M")
current_time_data
dirr ='HH_' + str(current_time_data.replace("/", "-").replace(":", "-"))
dirr = os.path.join(dirr1, dirr)

# Check if the subfolder exists, create if not
if not os.path.isdir(dirr):
    os.mkdir(dirr)
    print(f"Created subdirectory: {dirr}")
else:
    print("Subdirectory already exists.")

Created subdirectory: HH_2025-05-27_districts\HH_27-05-2025 13-45


In [6]:
# Happy Hour dates
date_from = ((pd.Timestamp.now()).replace(hour=21,minute=00).strftime('%#d/%#m/%Y %H:%M'))
date_to = ((pd.Timestamp.now() + pd.Timedelta(days = 1)).replace(hour=20,minute=59).strftime('%#d/%#m/%Y %H:%M'))
print(date_from,date_to)

27/5/2025 21:00 28/5/2025 20:59


In [7]:
# Offers date
given_time_str_2 = ((pd.Timestamp.now()).replace(hour=9,minute=0).strftime('%#d-%#m-%Y %H:%M'))
final_time_str_2 = ((pd.Timestamp.now() + pd.Timedelta(days = 1)).replace(hour=20,minute=59).strftime('%#d-%#m-%Y %H:%M'))
print(given_time_str_2,final_time_str_2)

27-5-2025 09:00 28-5-2025 20:59


# Queries

In [8]:
# top 20 skus per district
df1 = ret_metabase(42825)
df1.columns = df1.columns.str.lower()

In [9]:
# top skus per district high gross margin ,cm3 included
df2 = ret_metabase(43098)
df2.columns = df2.columns.str.lower()

In [10]:
# top skus per district high margin
df3 = ret_metabase(42784)
df3.columns = df3.columns.str.lower()

In [11]:
# active retailers
df4 = ret_metabase(36356)
df4.columns = df4.columns.str.lower()

In [12]:
# read packing units
df5 = ret_metabase(35788)
df5.columns = df5.columns.str.lower()

In [13]:
# retailers with delays
df6 = ret_metabase(28696)
df6.columns = df6.columns.str.lower()

In [14]:
#retailer bought skus
df7=ret_metabase(46923)
df7.columns = df7.columns.str.lower()
df7

,retailer_id,product_id,sku_freq
0,67,3,10.0
1,67,23,30.0
2,67,24,10.0
3,67,25,10.0
4,67,26,10.0
...,...,...,...
654171,873023,12352,100.0
654172,873023,12353,100.0
654173,873023,12373,100.0
654174,873023,12756,100.0


In [15]:
#sku info
df8=ret_metabase(27913)            #check margins
df8.columns = df8.columns.str.lower()

In [16]:
# top non food skus
df9 = ret_metabase(42783)
df9.columns = df9.columns.str.lower()

In [17]:
# # top non food skus WITHOUT LIMITATIONS
# df9 = ret_metabase(42826)
# df9.columns = df9.columns.str.lower()

In [18]:
# read retailer stocks
#Check inventory per warehouse
df10 = ret_metabase(1604)
df10.columns = df10.columns.str.lower()

In [19]:
# # read retailer dispaching rules for target supply chains 
# df11 = ret_metabase(8979)
# df11.columns = df11.columns.str.lower()

In [20]:
# retailers current wallet
df12 = ret_metabase(33544)
df12.columns=df12.columns.str.lower()

In [21]:
coupons = ret_metabase(47705)
coupons.columns = coupons.columns.str.lower()
coupons.rename({'value': 'wallet'}, axis = 1, inplace = True)

In [22]:
df12 = pd.concat([df12, coupons])

In [23]:
df12 = df12.groupby(['retailer_id'])['wallet'].sum().reset_index()

In [24]:
# retailers new segment 
df13 = ret_metabase(34193)
df13.columns=df13.columns.str.lower()

In [25]:
# retailer bought skus new 
df14 = ret_metabase(35452)
df14.columns=df14.columns.str.lower()

In [26]:
# skus with price constarints
df15 = ret_metabase(47961)
df15.columns=df15.columns.str.lower()

In [27]:
# min maxab price
df16 = ret_metabase(47962)
df16.columns=df16.columns.str.lower()

In [28]:
def change_columns_type(df):
    
    # convert column names to lower case
    df.columns = df.columns.str.lower()
    
    # create list that contains all column names 
    df_columns_names = df.columns
    
    # loop over columns to convert its types
    for column_name in df_columns_names:
        df[column_name] = df[column_name].astype(str)
        if df[column_name].str.contains('True', regex = False).any() | df[column_name].str.contains('False', regex = False).any():
            df[column_name] = df[column_name].astype(bool)
        else:   
            try:
                # check if the column contains '.' then convert columns to float else to int64
                if df[column_name].str.contains('.', regex=False).any():
                    df[column_name] = df[column_name].str.replace(',', '')
                    df[column_name] = df[column_name].astype(float)
                else:
                    df[column_name] = df[column_name].str.replace(',', '')
                    df[column_name] = df[column_name].astype('int64')
            except ValueError as e:
                pass
    return df

In [29]:
change_columns_type(df1)
change_columns_type(df2)
change_columns_type(df3)
change_columns_type(df4)
change_columns_type(df5)
change_columns_type(df6)
change_columns_type(df7)
change_columns_type(df8)
change_columns_type(df9)
change_columns_type(df10)
#change_columns_type(df11)
change_columns_type(df12)
change_columns_type(df13)
change_columns_type(df14)
change_columns_type(df15)
change_columns_type(df16)

,product_id,sku,maxab_price
0,11852,مولتو ميني ماجنم فراولة تشيز كيك - 10 جنية,204.25
1,9431,اوكسي جل يدوي نسيم الشرق- 70 جم,58.00
2,11434,لافندر مسحوق لافندر للغسالات العادية والفوق او...,170.00
3,11724,زينه مناديل رول مطبخ تريو 2 رول * 12 شرينك 2 بكرة,430.25
4,10472,اوكسي جل يدوي لافندر- 110 جم,44.25
...,...,...,...
5493,4223,بن عبد المعبود محوج وسط - 100 جم,782.50
5494,12721,ماكسى كولا - 1 لتر,99.25
5495,2194,اندومي لحم بيف سوبر مي عرض- 44 كيس 70 جم,265.75
5496,8939,ميرندا برتقال - 1.47 لتر,162.25


In [30]:
top_5_district_skus =df1.copy()
top_gross_skus = df2.copy()
top_margin_skus = df3.copy()
active_retailers = df4.copy()
packing_units = df5.copy()
retailer_delays = df6.copy()
retailer_bought_skus = df7.copy()
sku_info = df8.copy()
top_nonfood_skus = df9.copy()
retailer_stocks = df10.copy()
# retailer_dispaching_rules = df11.copy()
retailer_wallet = df12.copy()
retailer_segment = df13.copy()
retailer_bought_skus_new = df14.copy()
sku_constraints = df15.copy()
maxab_price = df16.copy()

In [31]:
len(top_nonfood_skus)

2280

# Retailer exclusions ***

In [ ]:
len(active_retailers)

In [ ]:
# aa=pd.read_clipboard()

In [ ]:
# active_retailers = active_retailers[active_retailers['retailer_id'].isin(aa['retailer_id'])]
# len(active_retailers)


In [ ]:
#remove retailrs with delays
active_retailers = active_retailers[~active_retailers['retailer_id'].isin(retailer_delays['retailer_id'])]
len(active_retailers)

In [ ]:
#merge retailers with df17 keeping only wallet
active_retailers = pd.merge(active_retailers, retailer_wallet[['retailer_id','wallet']], on='retailer_id', how='left')
#if wallet is null then 0
active_retailers['wallet'] = active_retailers['wallet'].fillna(0)
#remove rows with wallet >0
active_retailers = active_retailers[active_retailers['wallet']<20]
len(active_retailers)

In [ ]:
# temp_ret = pd.read_clipboard()              # @mai and magdy edit here   
# temp_ret.columns = temp_ret.columns.str.lower()
# active_retailers = active_retailers[active_retailers['retailer_id'].isin(temp_ret['retailer_id'])]

In [ ]:
# #keep only rows with retailer_id found in temp_ret
# active_retailers = active_retailers[active_retailers['retailer_id'].isin(temp_ret['retailer_id'])]
# len(active_retailers)

In [ ]:
# remove duplicates
active_retailers = active_retailers.drop_duplicates(subset=['retailer_id'])
len(active_retailers)

In [ ]:
# excluded retailers              @mai and magdy edit here
excluded = pd.read_clipboard()
excluded.columns = excluded.columns.str.lower()
# remove excluded retailers
active_retailers = active_retailers[~active_retailers['retailer_id'].isin(excluded['retailer_id'])]
len(active_retailers)

# Bucketizing retailers

In [32]:
#if segment is null fill with minimarket
active_retailers['segment'].fillna('minimarket',inplace=True)

In [33]:
# Calculate percentiles
percentile_10 = np.percentile(active_retailers['bm'], 10)
percentile_90 = np.percentile(active_retailers['bm'], 90)
percentile_10, percentile_90

(nan, nan)

In [34]:
retailers_margin_outliers = active_retailers[(active_retailers['bm'] < percentile_10) | (active_retailers['bm'] > percentile_90)]
retailers_margin_outliers

,retailer_id,region,district,segment,bm,orders,total_nmv,products


In [35]:
#remove retailers found in outliers
active_retailers = active_retailers[~active_retailers['retailer_id'].isin(retailers_margin_outliers['retailer_id'])]
len(active_retailers)

33623

In [36]:
#add new column avg_seg_bm which is the average bm per segment
active_retailers['avg_dis_bm']=active_retailers.groupby(['district'])['bm'].transform('mean')
active_retailers['avg_dis_bm']=active_retailers['avg_dis_bm'].round(4)
active_retailers

,retailer_id,region,district,segment,bm,orders,total_nmv,products,avg_dis_bm
0,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973
1,71,Greater Cairo,شبرا لخيمة 1,bulk,-22.828456,2,6523.8,14,-113.7973
2,73,Greater Cairo,شبرا لخيمة 1,minimarket,-34.602639,17,42242.1,131,-113.7973
3,76,Greater Cairo,شبرا لخيمة 2,minimarket,4.739166,1,1310.5,3,-80.7665
4,81,Greater Cairo,شبرا لخيمة 2,bulk,-51.311272,9,18644.9,35,-80.7665
...,...,...,...,...,...,...,...,...,...
33618,872901,Greater Cairo,العجوزة,nan,3.913569,1,1066.0,4,-94.1588
33619,872925,Greater Cairo,القناطر الخيرية,nan,12.615978,1,963.0,1,-90.1192
33620,872928,Greater Cairo,العجوزة,nan,15.415792,1,10700.0,1,-94.1588
33621,872951,Greater Cairo,مصر القديمة,nan,9.405185,1,6906.7,12,-113.5023


In [37]:
active_retailers['bm_diff']=(active_retailers['bm']-active_retailers['avg_dis_bm']).round(4)
active_retailers['z_score']=active_retailers['bm_diff']/active_retailers['avg_dis_bm'].round(4)
active_retailers

,retailer_id,region,district,segment,bm,orders,total_nmv,products,avg_dis_bm,bm_diff,z_score
0,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,-0.317356
1,71,Greater Cairo,شبرا لخيمة 1,bulk,-22.828456,2,6523.8,14,-113.7973,90.9688,-0.799393
2,73,Greater Cairo,شبرا لخيمة 1,minimarket,-34.602639,17,42242.1,131,-113.7973,79.1947,-0.695928
3,76,Greater Cairo,شبرا لخيمة 2,minimarket,4.739166,1,1310.5,3,-80.7665,85.5057,-1.058678
4,81,Greater Cairo,شبرا لخيمة 2,bulk,-51.311272,9,18644.9,35,-80.7665,29.4552,-0.364696
...,...,...,...,...,...,...,...,...,...,...,...
33618,872901,Greater Cairo,العجوزة,nan,3.913569,1,1066.0,4,-94.1588,98.0724,-1.041564
33619,872925,Greater Cairo,القناطر الخيرية,nan,12.615978,1,963.0,1,-90.1192,102.7352,-1.139992
33620,872928,Greater Cairo,العجوزة,nan,15.415792,1,10700.0,1,-94.1588,109.5746,-1.163721
33621,872951,Greater Cairo,مصر القديمة,nan,9.405185,1,6906.7,12,-113.5023,122.9075,-1.082864


In [38]:
#add new column ret_flag which is 'margin' if z_score is <-0.1
active_retailers['ret_flag']=np.where(active_retailers['z_score']<-0.1,'margin','')
active_retailers

,retailer_id,region,district,segment,bm,orders,total_nmv,products,avg_dis_bm,bm_diff,z_score,ret_flag
0,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,-0.317356,margin
1,71,Greater Cairo,شبرا لخيمة 1,bulk,-22.828456,2,6523.8,14,-113.7973,90.9688,-0.799393,margin
2,73,Greater Cairo,شبرا لخيمة 1,minimarket,-34.602639,17,42242.1,131,-113.7973,79.1947,-0.695928,margin
3,76,Greater Cairo,شبرا لخيمة 2,minimarket,4.739166,1,1310.5,3,-80.7665,85.5057,-1.058678,margin
4,81,Greater Cairo,شبرا لخيمة 2,bulk,-51.311272,9,18644.9,35,-80.7665,29.4552,-0.364696,margin
...,...,...,...,...,...,...,...,...,...,...,...,...
33618,872901,Greater Cairo,العجوزة,nan,3.913569,1,1066.0,4,-94.1588,98.0724,-1.041564,margin
33619,872925,Greater Cairo,القناطر الخيرية,nan,12.615978,1,963.0,1,-90.1192,102.7352,-1.139992,margin
33620,872928,Greater Cairo,العجوزة,nan,15.415792,1,10700.0,1,-94.1588,109.5746,-1.163721,margin
33621,872951,Greater Cairo,مصر القديمة,nan,9.405185,1,6906.7,12,-113.5023,122.9075,-1.082864,margin


In [39]:
#concat retailers_margin with retailers_margin_outliers 
active_retailers=pd.concat([active_retailers,retailers_margin_outliers],ignore_index=True)
active_retailers

,retailer_id,region,district,segment,bm,orders,total_nmv,products,avg_dis_bm,bm_diff,z_score,ret_flag
0,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,-0.317356,margin
1,71,Greater Cairo,شبرا لخيمة 1,bulk,-22.828456,2,6523.8,14,-113.7973,90.9688,-0.799393,margin
2,73,Greater Cairo,شبرا لخيمة 1,minimarket,-34.602639,17,42242.1,131,-113.7973,79.1947,-0.695928,margin
3,76,Greater Cairo,شبرا لخيمة 2,minimarket,4.739166,1,1310.5,3,-80.7665,85.5057,-1.058678,margin
4,81,Greater Cairo,شبرا لخيمة 2,bulk,-51.311272,9,18644.9,35,-80.7665,29.4552,-0.364696,margin
...,...,...,...,...,...,...,...,...,...,...,...,...
33618,872901,Greater Cairo,العجوزة,nan,3.913569,1,1066.0,4,-94.1588,98.0724,-1.041564,margin
33619,872925,Greater Cairo,القناطر الخيرية,nan,12.615978,1,963.0,1,-90.1192,102.7352,-1.139992,margin
33620,872928,Greater Cairo,العجوزة,nan,15.415792,1,10700.0,1,-94.1588,109.5746,-1.163721,margin
33621,872951,Greater Cairo,مصر القديمة,nan,9.405185,1,6906.7,12,-113.5023,122.9075,-1.082864,margin


In [40]:
#if bm less than 4.1 then flag as 'margin'
active_retailers['ret_flag']=np.where(active_retailers['bm']<4.1,'margin',active_retailers['ret_flag'])
active_retailers

,retailer_id,region,district,segment,bm,orders,total_nmv,products,avg_dis_bm,bm_diff,z_score,ret_flag
0,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,-0.317356,margin
1,71,Greater Cairo,شبرا لخيمة 1,bulk,-22.828456,2,6523.8,14,-113.7973,90.9688,-0.799393,margin
2,73,Greater Cairo,شبرا لخيمة 1,minimarket,-34.602639,17,42242.1,131,-113.7973,79.1947,-0.695928,margin
3,76,Greater Cairo,شبرا لخيمة 2,minimarket,4.739166,1,1310.5,3,-80.7665,85.5057,-1.058678,margin
4,81,Greater Cairo,شبرا لخيمة 2,bulk,-51.311272,9,18644.9,35,-80.7665,29.4552,-0.364696,margin
...,...,...,...,...,...,...,...,...,...,...,...,...
33618,872901,Greater Cairo,العجوزة,nan,3.913569,1,1066.0,4,-94.1588,98.0724,-1.041564,margin
33619,872925,Greater Cairo,القناطر الخيرية,nan,12.615978,1,963.0,1,-90.1192,102.7352,-1.139992,margin
33620,872928,Greater Cairo,العجوزة,nan,15.415792,1,10700.0,1,-94.1588,109.5746,-1.163721,margin
33621,872951,Greater Cairo,مصر القديمة,nan,9.405185,1,6906.7,12,-113.5023,122.9075,-1.082864,margin


In [41]:
# replace empty and NULL strings with 'normal'
active_retailers['ret_flag']=active_retailers['ret_flag'].replace('', 'normal')
active_retailers['ret_flag']=active_retailers['ret_flag'].fillna('normal')
active_retailers


,retailer_id,region,district,segment,bm,orders,total_nmv,products,avg_dis_bm,bm_diff,z_score,ret_flag
0,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,-0.317356,margin
1,71,Greater Cairo,شبرا لخيمة 1,bulk,-22.828456,2,6523.8,14,-113.7973,90.9688,-0.799393,margin
2,73,Greater Cairo,شبرا لخيمة 1,minimarket,-34.602639,17,42242.1,131,-113.7973,79.1947,-0.695928,margin
3,76,Greater Cairo,شبرا لخيمة 2,minimarket,4.739166,1,1310.5,3,-80.7665,85.5057,-1.058678,margin
4,81,Greater Cairo,شبرا لخيمة 2,bulk,-51.311272,9,18644.9,35,-80.7665,29.4552,-0.364696,margin
...,...,...,...,...,...,...,...,...,...,...,...,...
33618,872901,Greater Cairo,العجوزة,nan,3.913569,1,1066.0,4,-94.1588,98.0724,-1.041564,margin
33619,872925,Greater Cairo,القناطر الخيرية,nan,12.615978,1,963.0,1,-90.1192,102.7352,-1.139992,margin
33620,872928,Greater Cairo,العجوزة,nan,15.415792,1,10700.0,1,-94.1588,109.5746,-1.163721,margin
33621,872951,Greater Cairo,مصر القديمة,nan,9.405185,1,6906.7,12,-113.5023,122.9075,-1.082864,margin


In [42]:
#print uniqe values in ret_flag
active_retailers['ret_flag'].unique()

array(['margin', 'normal'], dtype=object)

In [43]:
## Add wh
ret_warehouse = ret_metabase(56815)
ret_warehouse = change_columns_type(ret_warehouse)
active_retailers = active_retailers.merge(ret_warehouse,on='retailer_id')

# Data merge

In [44]:
retailer_segment

,retailer_id,region,segment_new,segment
0,292364,Greater Cairo,food,nan
1,26487,Greater Cairo,food,supermarket
2,187679,Greater Cairo,food,minimarket
3,69842,Greater Cairo,nonfood,minimarket
4,588313,Greater Cairo,food,nan
...,...,...,...,...
39827,292945,Greater Cairo,nonfood,monazzefat
39828,734607,Greater Cairo,food,nan
39829,200113,Greater Cairo,food,bulk
39830,37731,Greater Cairo,food,koshk


In [45]:
#merge active_retailers with retailer_segment keeping only segment_new
active_retailers = pd.merge(active_retailers, retailer_segment[['retailer_id','segment_new']], on='retailer_id', how='left')
active_retailers.head()

,retailer_id,region,district,segment,bm,orders,total_nmv,products,avg_dis_bm,bm_diff,z_score,ret_flag,wh_id,warehouse,segment_new
0,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,-0.317356,margin,1,Mostorod,food
1,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,-0.317356,margin,38,El-Marg,food
2,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,-0.317356,margin,467,Saryaqus Frozen 2 pl,food
3,71,Greater Cairo,شبرا لخيمة 1,bulk,-22.828456,2,6523.8,14,-113.7973,90.9688,-0.799393,margin,1,Mostorod,food
4,73,Greater Cairo,شبرا لخيمة 1,minimarket,-34.602639,17,42242.1,131,-113.7973,79.1947,-0.695928,margin,467,Saryaqus Frozen 2 pl,food


In [46]:
len(active_retailers)

55954

In [47]:
# non_food_ret = rows where segment is monazzefat or papershop or pharmacy
non_food_ret = active_retailers[active_retailers['segment_new'].isin(['nonfood'])]
len(non_food_ret)

5630

In [48]:
# food_ret = rows where retailer_id is not in non_food_ret
food_ret = active_retailers[~active_retailers['retailer_id'].isin(non_food_ret['retailer_id'])]
len(food_ret)

50324

In [49]:
food_ret

,retailer_id,region,district,segment,bm,orders,total_nmv,products,avg_dis_bm,bm_diff,z_score,ret_flag,wh_id,warehouse,segment_new
0,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,-0.317356,margin,1,Mostorod,food
1,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,-0.317356,margin,38,El-Marg,food
2,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,-0.317356,margin,467,Saryaqus Frozen 2 pl,food
3,71,Greater Cairo,شبرا لخيمة 1,bulk,-22.828456,2,6523.8,14,-113.7973,90.9688,-0.799393,margin,1,Mostorod,food
4,73,Greater Cairo,شبرا لخيمة 1,minimarket,-34.602639,17,42242.1,131,-113.7973,79.1947,-0.695928,margin,467,Saryaqus Frozen 2 pl,food
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55949,872901,Greater Cairo,العجوزة,nan,3.913569,1,1066.0,4,-94.1588,98.0724,-1.041564,margin,39,Basatin,food
55950,872925,Greater Cairo,القناطر الخيرية,nan,12.615978,1,963.0,1,-90.1192,102.7352,-1.139992,margin,1,Mostorod,food
55951,872928,Greater Cairo,العجوزة,nan,15.415792,1,10700.0,1,-94.1588,109.5746,-1.163721,margin,39,Basatin,food
55952,872951,Greater Cairo,مصر القديمة,nan,9.405185,1,6906.7,12,-113.5023,122.9075,-1.082864,margin,39,Basatin,food


In [50]:
#merge food_ret with top_5_district_skus on district
ret_sku_no_limitations = pd.merge(food_ret, top_5_district_skus, on='district', how='left')
ret_sku_no_limitations

,retailer_id,region,district,segment,bm,orders_x,total_nmv,products,avg_dis_bm,bm_diff,...,category,brand_id,brand_name,product_id,sku,nmv,orders_y,quantity,retailers,sku_rank
0,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,...,عصاير,35.0,بيتي عصاير,143.0,عصير بيتى تفاح - 235 مل,143439.0,64.0,780.0,59.0,1.0
1,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,...,عصاير,35.0,بيتي عصاير,144.0,عصير بيتى كوكتيل - 235 مل,155509.2,64.0,846.0,57.0,2.0
2,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,...,عصاير,35.0,بيتي عصاير,305.0,عصير بيتى مانجو - 235 مل,114697.8,46.0,624.0,45.0,3.0
3,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,...,حاجه ساقعه,33.0,بيبسي,589.0,بيبسى - 2.43 لتر,295225.2,45.0,1548.0,40.0,4.0
4,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,...,عصاير,293.0,بست,1777.0,عصير بست تفاح كيس - 190 مل,86851.2,42.0,408.0,40.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006019,873023,Greater Cairo,الجيزة,nan,5.598434,1,6444.0,49,-93.2892,98.8876,...,حاجه ساقعه,33.0,بيبسي,1126.0,ميرندا برتقال تربو - 390 مل,53028.0,23.0,420.0,21.0,16.0
1006020,873023,Greater Cairo,الجيزة,nan,5.598434,1,6444.0,49,-93.2892,98.8876,...,قهوة,19.0,نسكافية,62.0,نسكافية بلاك ظرف- 1.8 جم,88990.5,22.0,522.0,21.0,17.0
1006021,873023,Greater Cairo,الجيزة,nan,5.598434,1,6444.0,49,-93.2892,98.8876,...,عصاير,35.0,بيتي عصاير,148.0,عصير بيتى كوكتيل - 1 لتر,54591.0,20.0,168.0,20.0,18.0
1006022,873023,Greater Cairo,الجيزة,nan,5.598434,1,6444.0,49,-93.2892,98.8876,...,عصاير,35.0,بيتي عصاير,149.0,عصير بيتى جوافة - 1 لتر,50908.5,20.0,156.0,20.0,19.0


In [51]:
retailer_bought_skus['flag']=1
retailer_bought_skus

,retailer_id,product_id,sku_freq,flag
0,67,3,10.0,1
1,67,23,30.0,1
2,67,24,10.0,1
3,67,25,10.0,1
4,67,26,10.0,1
...,...,...,...,...
654171,873023,12352,100.0,1
654172,873023,12353,100.0,1
654173,873023,12373,100.0,1
654174,873023,12756,100.0,1


In [52]:
# x = ret_sku_no_limitations.merge(retailer_bought_skus[['retailer_id','product_id','flag']], on=['retailer_id','product_id'], how='left')
# ret_sku_no_limitations = x[x['flag']!=1]
# ret_sku_no_limitations

In [53]:
#per retailer_id select only the top 5 skus
ret_sku_no_limitations = ret_sku_no_limitations.groupby('retailer_id').head(5)
ret_sku_no_limitations


,retailer_id,region,district,segment,bm,orders_x,total_nmv,products,avg_dis_bm,bm_diff,...,category,brand_id,brand_name,product_id,sku,nmv,orders_y,quantity,retailers,sku_rank
0,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,...,عصاير,35.0,بيتي عصاير,143.0,عصير بيتى تفاح - 235 مل,143439.000,64.0,780.0,59.0,1.0
1,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,...,عصاير,35.0,بيتي عصاير,144.0,عصير بيتى كوكتيل - 235 مل,155509.200,64.0,846.0,57.0,2.0
2,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,...,عصاير,35.0,بيتي عصاير,305.0,عصير بيتى مانجو - 235 مل,114697.800,46.0,624.0,45.0,3.0
3,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,...,حاجه ساقعه,33.0,بيبسي,589.0,بيبسى - 2.43 لتر,295225.200,45.0,1548.0,40.0,4.0
4,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,...,عصاير,293.0,بست,1777.0,عصير بست تفاح كيس - 190 مل,86851.200,42.0,408.0,40.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006004,873023,Greater Cairo,الجيزة,nan,5.598434,1,6444.0,49,-93.2892,98.8876,...,عصاير,35.0,بيتي عصاير,305.0,عصير بيتى مانجو - 235 مل,114121.632,54.0,624.0,50.0,1.0
1006005,873023,Greater Cairo,الجيزة,nan,5.598434,1,6444.0,49,-93.2892,98.8876,...,عصاير,35.0,بيتي عصاير,144.0,عصير بيتى كوكتيل - 235 مل,93214.500,45.0,510.0,42.0,2.0
1006006,873023,Greater Cairo,الجيزة,nan,5.598434,1,6444.0,49,-93.2892,98.8876,...,حاجه ساقعه,33.0,بيبسي,326.0,بيبسى كانز ستار - 320 مل,469758.000,44.0,1440.0,42.0,3.0
1006007,873023,Greater Cairo,الجيزة,nan,5.598434,1,6444.0,49,-93.2892,98.8876,...,عصاير,35.0,بيتي عصاير,143.0,عصير بيتى تفاح - 235 مل,83446.500,40.0,456.0,39.0,4.0


In [54]:
# export to excel
ret_sku_no_limitations.to_excel(dirr+'/ret_sku_no_limitations.xlsx', index=False)

In [55]:
#print number of unique retailers
len(ret_sku_no_limitations['retailer_id'].unique())

29305

In [56]:
#divide food_ret into 2 dataframes based on ret_flag
food_margin_ret = food_ret[food_ret['ret_flag']=='margin']
food_normal_ret = food_ret[food_ret['ret_flag']=='normal']
len(food_margin_ret), len(food_normal_ret)

(50323, 1)

In [57]:
gross_skus=pd.concat([top_gross_skus],ignore_index=True)
margin_skus=pd.concat([top_margin_skus],ignore_index=True)
#remove rows with duplicate district and product_id in both data frames
gross_skus = gross_skus.drop_duplicates(subset=['district','product_id'])
margin_skus = margin_skus.drop_duplicates(subset=['district','product_id'])
len(gross_skus), len(margin_skus)

(1920, 1920)

In [58]:
top_gross_skus

,district,category,brand_id,brand_name,product_id,sku,nmv,orders,quantity,retailers,sku_rank
0,قليوب سنتر,عصاير,35,بيتي عصاير,144,عصير بيتى كوكتيل - 235 مل,11540.50,21,63,21,1
1,قليوب سنتر,عصاير,35,بيتي عصاير,143,عصير بيتى تفاح - 235 مل,4564.50,15,25,15,2
2,قليوب سنتر,عصاير,35,بيتي عصاير,305,عصير بيتى مانجو - 235 مل,4198.75,13,23,13,3
3,قليوب سنتر,حاجه ساقعه,33,بيبسي,589,بيبسى - 2.43 لتر,6735.25,12,35,12,4
4,قليوب سنتر,منظفات,90,كلوركس,9372,كلوركس الوان باوش 5 جنيه - 100 مل,5382.00,12,52,12,5
...,...,...,...,...,...,...,...,...,...,...,...
1915,بولاق الدكرور,عصاير,293,بست,1777,عصير بست تفاح كيس - 190 مل,12900.00,33,60,33,26
1916,بولاق الدكرور,صلصة و صوص,45,هاينز,9507,كاتشب هاينز الكبير - 16 جم,13694.67,33,54,32,27
1917,بولاق الدكرور,صلصة و صوص,15,هارفست فوودز,346,هارفست صلصة ظرف - 30 جم,6805.50,30,60,30,28
1918,بولاق الدكرور,خل,183,حبوبة,74,حبوبة خل- 900 مل,4879.00,29,34,29,29


In [59]:
food_ret.to_excel(dirr+'/food_ret.xlsx',index=False)
non_food_ret.to_excel(dirr+'/non_food_ret.xlsx',index=False)

In [60]:
# retailer_dispaching_rules = retailer_dispaching_rules[['retailer_id','warehouse_id']]

In [61]:
# #merge non_food_ret with retailer_dispaching_rules
# non_food_ret = pd.merge(non_food_ret, retailer_dispaching_rules, on='retailer_id', how='left')
# #merge food_ret with retailer_dispaching_rules
# food_ret = pd.merge(food_ret, retailer_dispaching_rules, on='retailer_id', how='left')

In [62]:
# #remove rows with warehouse_id is null
# non_food_ret = non_food_ret[non_food_ret['warehouse_id'].notnull()]
# food_ret = food_ret[food_ret['warehouse_id'].notnull()]

In [63]:
top_nonfood_skus

,district,category,brand_id,brand_name,product_id,sku,nmv,orders,quantity,retailers,sku_rank
0,15 مايو,بطاريات ولمبات,2252,توشيبا,12677,AA توشيبا ازرق 4 حجر قلم شرنك 4 حجر,1010.0,1,4,1,1
1,15 مايو,بطاريات ولمبات,2252,توشيبا,12676,AAA توشيبا ازرق 2 حجر ريموت شرنك 2 حجر,1007.0,1,4,1,2
2,15 مايو,منظفات,396,باهي,4253,باهي مسحوق يدوي بالافندر - 1000 جم,1002.0,1,6,1,3
3,15 مايو,حفاضات أطفال,356,جود كير,6098,حفاضات جود كير مقاس 4 - 100 حفاضة,749.0,1,2,1,4
4,15 مايو,منظفات,72,برسيل,12638,برسيل يدوى لافندر عرض كيس زيادة - 110 جم,522.0,1,4,1,5
...,...,...,...,...,...,...,...,...,...,...,...
2275,وسط امبابة,ورقيات,290,بابيا,9289,بابيا مناديل وجه نعومة و نقاء 3 طبقة برميم - 5...,1812.0,2,4,2,36
2276,وسط امبابة,ورقيات,344,زينه,9760,زينة تريو عرض - 500 منديل,1614.0,2,4,2,37
2277,وسط امبابة,حفاضات أطفال,260,مولفيكس,11558,مولفيكس بانتس ميجا مقاس 4 - 80 حفاضة 80 حفاضة,1613.0,2,4,2,38
2278,وسط امبابة,كريم و جل للشعر,2078,هيربسينديا,10983,هيربسينديا زيت شعر اعشاب الهند بالروزمارى و7بذ...,1352.0,2,4,2,39


In [64]:
#merge non_food_ret with non_food_skus
non_food_ret_sku = pd.merge(non_food_ret, top_nonfood_skus, on='district', how='left')
#merge food_margin_ret with margin_skus
food_margin_ret_sku = pd.merge(food_margin_ret, gross_skus, on='district', how='left')
#merge food_normal_ret with gross_skus
food_normal_ret_sku = pd.merge(food_normal_ret, margin_skus , on='district', how='left')

In [65]:
top_nonfood_skus.to_excel(dirr+'/top_non_food_skus.xlsx',index=False)
margin_skus.to_excel(dirr+'/margin_skus.xlsx',index=False)
gross_skus.to_excel(dirr+'/gross_skus.xlsx',index=False)

In [66]:
len(non_food_ret_sku)

224701

In [67]:
food_margin_ret_sku

,retailer_id,region,district,segment,bm,orders_x,total_nmv,products,avg_dis_bm,bm_diff,...,category,brand_id,brand_name,product_id,sku,nmv,orders_y,quantity,retailers,sku_rank
0,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,...,عصاير,35.0,بيتي عصاير,143.0,عصير بيتى تفاح - 235 مل,23906.50,64.0,130.0,59.0,1.0
1,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,...,عصاير,35.0,بيتي عصاير,144.0,عصير بيتى كوكتيل - 235 مل,25918.20,64.0,141.0,57.0,2.0
2,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,...,عصاير,35.0,بيتي عصاير,305.0,عصير بيتى مانجو - 235 مل,19116.30,46.0,104.0,45.0,3.0
3,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,...,حاجه ساقعه,33.0,بيبسي,589.0,بيبسى - 2.43 لتر,24602.10,45.0,129.0,40.0,4.0
4,67,Greater Cairo,شبرا لخيمة 1,minimarket,-77.683129,34,54812.2,119,-113.7973,36.1142,...,عصاير,293.0,بست,1777.0,عصير بست تفاح كيس - 190 مل,14475.20,42.0,68.0,40.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1509018,873023,Greater Cairo,الجيزة,nan,5.598434,1,6444.0,49,-93.2892,98.8876,...,مرقة وخلطات,26.0,ماجي,161.0,ماجي مرقة دجاج شريط- 8 جم,6692.50,18.0,49.0,17.0,26.0
1509019,873023,Greater Cairo,الجيزة,nan,5.598434,1,6444.0,49,-93.2892,98.8876,...,ألبان,42.0,بخيره,130.0,بخيره لبن- 500 مل,15745.50,17.0,34.0,16.0,27.0
1509020,873023,Greater Cairo,الجيزة,nan,5.598434,1,6444.0,49,-93.2892,98.8876,...,شاي,4.0,العروسة,7886.0,شاى العروسة - 250 جم,4932.00,16.0,24.0,16.0,28.0
1509021,873023,Greater Cairo,الجيزة,nan,5.598434,1,6444.0,49,-93.2892,98.8876,...,حاجه ساقعه,33.0,بيبسي,1446.0,سفن اب - 2.43 لتر,4429.75,16.0,23.0,16.0,29.0


In [68]:
ret_skus=pd.concat([non_food_ret_sku,food_normal_ret_sku,food_margin_ret_sku,ret_sku_no_limitations],ignore_index=True) #@mai and magdy edit here
# ret_skus=pd.concat([non_food_ret_sku,food_normal_ret_sku,food_margin_ret_sku],ignore_index=True)
ret_skus

,retailer_id,region,district,segment,bm,orders_x,total_nmv,products,avg_dis_bm,bm_diff,...,category,brand_id,brand_name,product_id,sku,nmv,orders_y,quantity,retailers,sku_rank
0,2062,Greater Cairo,شبرا لخيمة 1,monazzefat,-924.323252,1,2024.8,10,-113.7973,-810.5260,...,حفاضات أطفال,292.0,هلو بيبي,1767.0,هلو بيبي ماكسي مقاس5- 40 حفاضة,20927.400,18.0,60.0,18.0,1.0
1,2062,Greater Cairo,شبرا لخيمة 1,monazzefat,-924.323252,1,2024.8,10,-113.7973,-810.5260,...,شامبو و شاور جيل,285.0,سباركل,6140.0,سباركل شامبو مشكل + 10 جنيه بضاعه مجانيه - 40...,11781.000,17.0,42.0,17.0,2.0
2,2062,Greater Cairo,شبرا لخيمة 1,monazzefat,-924.323252,1,2024.8,10,-113.7973,-810.5260,...,حفاضات أطفال,356.0,جود كير,6098.0,حفاضات جود كير مقاس 4 - 100 حفاضة,27458.600,14.0,42.0,14.0,3.0
3,2062,Greater Cairo,شبرا لخيمة 1,monazzefat,-924.323252,1,2024.8,10,-113.7973,-810.5260,...,حفاضات أطفال,292.0,هلو بيبي,1766.0,هلو بيبي كبير مقاس 4- 40 حفاضة,19564.200,14.0,46.0,14.0,4.0
4,2062,Greater Cairo,شبرا لخيمة 1,monazzefat,-924.323252,1,2024.8,10,-113.7973,-810.5260,...,حفاضات أطفال,356.0,جود كير,6097.0,حفاضات جود كير مقاس 5 - 100 حفاضة,40958.400,12.0,58.0,12.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880149,873023,Greater Cairo,الجيزة,nan,5.598434,1,6444.0,49,-93.2892,98.8876,...,عصاير,35.0,بيتي عصاير,305.0,عصير بيتى مانجو - 235 مل,114121.632,54.0,624.0,50.0,1.0
1880150,873023,Greater Cairo,الجيزة,nan,5.598434,1,6444.0,49,-93.2892,98.8876,...,عصاير,35.0,بيتي عصاير,144.0,عصير بيتى كوكتيل - 235 مل,93214.500,45.0,510.0,42.0,2.0
1880151,873023,Greater Cairo,الجيزة,nan,5.598434,1,6444.0,49,-93.2892,98.8876,...,حاجه ساقعه,33.0,بيبسي,326.0,بيبسى كانز ستار - 320 مل,469758.000,44.0,1440.0,42.0,3.0
1880152,873023,Greater Cairo,الجيزة,nan,5.598434,1,6444.0,49,-93.2892,98.8876,...,عصاير,35.0,بيتي عصاير,143.0,عصير بيتى تفاح - 235 مل,83446.500,40.0,456.0,39.0,4.0


In [69]:
retailer_bought_skus_new

,retailer_id,product_id,sku_freq
0,67,3,10.0
1,67,23,20.0
2,67,24,10.0
3,67,33,10.0
4,67,57,20.0
...,...,...,...
295758,873023,12349,100.0
295759,873023,12350,100.0
295760,873023,12353,100.0
295761,873023,12756,100.0


In [70]:
retailer_bought_skus_new = retailer_bought_skus_new[retailer_bought_skus_new['sku_freq']>90]
retailer_bought_skus_new['flag_2']=1
retailer_bought_skus_new

,retailer_id,product_id,sku_freq,flag_2
22,71,25,100.0,1
23,71,33,100.0,1
24,71,431,100.0,1
25,71,7885,100.0,1
26,71,7886,100.0,1
...,...,...,...,...
295758,873023,12349,100.0,1
295759,873023,12350,100.0,1
295760,873023,12353,100.0,1
295761,873023,12756,100.0,1


In [71]:
len(ret_skus)

1880154

In [72]:
y = ret_skus.merge(retailer_bought_skus_new[['retailer_id','product_id','flag_2']], on=['retailer_id','product_id'], how='left')
ret_temp = y[y['flag_2']!=1]


In [73]:
ret_skus=ret_temp

In [74]:
len(ret_skus)

1856153

In [75]:
food_margin_ret_sku['retailer_id'].nunique(),food_normal_ret_sku['retailer_id'].nunique(), non_food_ret_sku['retailer_id'].nunique()

(29304, 1, 4267)

In [76]:
#keep only retailer_id ,district, product_id, sku, warehouse_id, segment
ret_skus = ret_skus[['retailer_id','district','product_id','category','brand_name','sku','segment']]

In [77]:
ret_skus['retailer_id'].nunique()

33572

In [78]:
#drop rows with duplicate retailer_id and product_id
ret_skus = ret_skus.drop_duplicates(subset=['retailer_id','product_id'])
len(ret_skus)

1036217

In [79]:
#drop rows with product_id is null
ret_skus = ret_skus[ret_skus['product_id'].notnull()]
len(ret_skus)

1036192

In [80]:
# keep only rows from sku_info where region = Cairo or Giza
sku_info = sku_info[sku_info['region'].isin(['Cairo','Giza'])]
sku_info

,region,packing_unit_id,sku_id,section,cat,brand,sku,wac1,wac_p,price,fm,bm,rnk
2,Cairo,2,3,مأكولات أساسية,أرز,حبوبة,ارز حبوبة رفيع - 1 كجم,263.597351,258.188473,271.75,0.030001,0.049904,1
7,Giza,2,3,مأكولات أساسية,أرز,حبوبة,ارز حبوبة رفيع - 1 كجم,263.597351,258.188473,271.75,0.030001,0.049904,1
10,Cairo,1,5,مأكولات أساسية,زيوت,كريستال,كريستال زيت ذرة 12 أزازة Delisted- 900 مل,340.955206,332.740000,374.50,0.089572,0.111509,1
15,Cairo,1,6,مأكولات أساسية,زيوت,كريستال,كريستال زيت ذرة 6 أزازة Delisted- 2.4 لتر,449.468097,432.520000,492.00,0.086447,0.120894,1
20,Cairo,1,7,مأكولات أساسية,زيوت,كريستال,زيت كريستال عباد الشمس (12 أزازة - عرض 20 جنية...,221.617153,221.620000,366.30,0.394985,0.394977,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54136,Giza,2,19954,عناية شخصية,كريم و جل للشعر,دابر املا,زيت شعر دابر املا و زيت الزيتون و اللوز للاطفا...,484.842500,470.297225,510.00,0.049328,0.077849,1
54140,Cairo,3,19971,حلويات وسناكس,بسكويت و معمول,توك,بسكويت توك بالملح 2+6 عرض - 30.5 جرام,80.000000,73.600000,79.25,-0.009464,0.071293,1
54145,Giza,3,19971,حلويات وسناكس,بسكويت و معمول,توك,بسكويت توك بالملح 2+6 عرض - 30.5 جرام,80.000000,73.600000,78.00,-0.025641,0.056410,1
54149,Cairo,3,19972,حلويات وسناكس,بسكويت و معمول,توك,بسكويت توك بالجبنة عرض 2+6 - 32 جرام,80.000000,73.600000,79.25,-0.009464,0.071293,1


In [81]:
# group by product_id and get the average bm
sku_info = sku_info.groupby('sku_id').agg({'bm':'mean'}).reset_index()
sku_info

,sku_id,bm
0,3,0.049904
1,5,0.111509
2,6,0.120894
3,7,0.394977
4,8,0.174316
...,...,...
7348,19949,0.111180
7349,19951,0.114329
7350,19954,0.064834
7351,19971,0.063852


In [82]:
#rename sku_id to product_id
sku_info.rename(columns={'sku_id':'product_id'},inplace=True)

In [83]:
#join ret_skus with sku_info on product_id to get the bm
ret_skus = pd.merge(ret_skus, sku_info, on = 'product_id', how='left')
ret_skus

,retailer_id,district,product_id,category,brand_name,sku,segment,bm
0,2062,شبرا لخيمة 1,1767.0,حفاضات أطفال,هلو بيبي,هلو بيبي ماكسي مقاس5- 40 حفاضة,monazzefat,0.036667
1,2062,شبرا لخيمة 1,6140.0,شامبو و شاور جيل,سباركل,سباركل شامبو مشكل + 10 جنيه بضاعه مجانيه - 40...,monazzefat,0.041606
2,2062,شبرا لخيمة 1,6098.0,حفاضات أطفال,جود كير,حفاضات جود كير مقاس 4 - 100 حفاضة,monazzefat,0.040251
3,2062,شبرا لخيمة 1,1766.0,حفاضات أطفال,هلو بيبي,هلو بيبي كبير مقاس 4- 40 حفاضة,monazzefat,0.042029
4,2062,شبرا لخيمة 1,6097.0,حفاضات أطفال,جود كير,حفاضات جود كير مقاس 5 - 100 حفاضة,monazzefat,0.043363
...,...,...,...,...,...,...,...,...
1036187,863117,النزهه,5153.0,مشروبات الطاقة,ريد بل,ريد بل ابيض بجوز الهند و التوت - 250 مل,nan,0.055057
1036188,863498,مدينة نصر 2,5153.0,مشروبات الطاقة,ريد بل,ريد بل ابيض بجوز الهند و التوت - 250 مل,nan,0.055057
1036189,864889,النزهه,5153.0,مشروبات الطاقة,ريد بل,ريد بل ابيض بجوز الهند و التوت - 250 مل,nan,0.055057
1036190,865741,النزهه,5153.0,مشروبات الطاقة,ريد بل,ريد بل ابيض بجوز الهند و التوت - 250 مل,nan,0.055057


In [84]:
#drop rows with bm = null
ret_skus = ret_skus[ret_skus['bm'].notnull()]
ret_skus

,retailer_id,district,product_id,category,brand_name,sku,segment,bm
0,2062,شبرا لخيمة 1,1767.0,حفاضات أطفال,هلو بيبي,هلو بيبي ماكسي مقاس5- 40 حفاضة,monazzefat,0.036667
1,2062,شبرا لخيمة 1,6140.0,شامبو و شاور جيل,سباركل,سباركل شامبو مشكل + 10 جنيه بضاعه مجانيه - 40...,monazzefat,0.041606
2,2062,شبرا لخيمة 1,6098.0,حفاضات أطفال,جود كير,حفاضات جود كير مقاس 4 - 100 حفاضة,monazzefat,0.040251
3,2062,شبرا لخيمة 1,1766.0,حفاضات أطفال,هلو بيبي,هلو بيبي كبير مقاس 4- 40 حفاضة,monazzefat,0.042029
4,2062,شبرا لخيمة 1,6097.0,حفاضات أطفال,جود كير,حفاضات جود كير مقاس 5 - 100 حفاضة,monazzefat,0.043363
...,...,...,...,...,...,...,...,...
1036187,863117,النزهه,5153.0,مشروبات الطاقة,ريد بل,ريد بل ابيض بجوز الهند و التوت - 250 مل,nan,0.055057
1036188,863498,مدينة نصر 2,5153.0,مشروبات الطاقة,ريد بل,ريد بل ابيض بجوز الهند و التوت - 250 مل,nan,0.055057
1036189,864889,النزهه,5153.0,مشروبات الطاقة,ريد بل,ريد بل ابيض بجوز الهند و التوت - 250 مل,nan,0.055057
1036190,865741,النزهه,5153.0,مشروبات الطاقة,ريد بل,ريد بل ابيض بجوز الهند و التوت - 250 مل,nan,0.055057


In [85]:
#discount_adj = pd.read_clipboard()

In [86]:
#discount_adj

In [87]:
#ret_skus = ret_skus.merge(discount_adj,how='left',on='retailer_id').fillna(0.4)

In [88]:
#add new column discount = blended_margin * 0.3 rounded to 4 decimals and floor =0.01 and ceil = 0.03  
ret_skus['discount'] = ret_skus['bm']*0.55         # burn sensetivty exp @mai and magdy edit here
ret_skus['discount'] = ret_skus['discount'].round(3)
ret_skus['discount'] = ret_skus['discount'].apply(lambda x: max(x,0.01))
ret_skus['discount'] = ret_skus['discount'].apply(lambda x: min(x,0.025))      #magdy and mai edit here
ret_skus['discount']=ret_skus['discount']*100

In [ ]:
ret_skus

In [ ]:
# #if retailer_id is in non_food_ret then discount = discount + 0.5
# mask = ret_skus['retailer_id'].isin(non_food_ret['retailer_id'])
# ret_skus.loc[mask, 'discount'] += 0.25


In [89]:
#length of unique retailers
ret_skus['retailer_id'].nunique()

33547

In [90]:
ret_skus

,retailer_id,district,product_id,category,brand_name,sku,segment,bm,discount
0,2062,شبرا لخيمة 1,1767.0,حفاضات أطفال,هلو بيبي,هلو بيبي ماكسي مقاس5- 40 حفاضة,monazzefat,0.036667,2.0
1,2062,شبرا لخيمة 1,6140.0,شامبو و شاور جيل,سباركل,سباركل شامبو مشكل + 10 جنيه بضاعه مجانيه - 40...,monazzefat,0.041606,2.3
2,2062,شبرا لخيمة 1,6098.0,حفاضات أطفال,جود كير,حفاضات جود كير مقاس 4 - 100 حفاضة,monazzefat,0.040251,2.2
3,2062,شبرا لخيمة 1,1766.0,حفاضات أطفال,هلو بيبي,هلو بيبي كبير مقاس 4- 40 حفاضة,monazzefat,0.042029,2.3
4,2062,شبرا لخيمة 1,6097.0,حفاضات أطفال,جود كير,حفاضات جود كير مقاس 5 - 100 حفاضة,monazzefat,0.043363,2.4
...,...,...,...,...,...,...,...,...,...
1036187,863117,النزهه,5153.0,مشروبات الطاقة,ريد بل,ريد بل ابيض بجوز الهند و التوت - 250 مل,nan,0.055057,2.5
1036188,863498,مدينة نصر 2,5153.0,مشروبات الطاقة,ريد بل,ريد بل ابيض بجوز الهند و التوت - 250 مل,nan,0.055057,2.5
1036189,864889,النزهه,5153.0,مشروبات الطاقة,ريد بل,ريد بل ابيض بجوز الهند و التوت - 250 مل,nan,0.055057,2.5
1036190,865741,النزهه,5153.0,مشروبات الطاقة,ريد بل,ريد بل ابيض بجوز الهند و التوت - 250 مل,nan,0.055057,2.5


In [91]:
ret_1 = ret_skus[['retailer_id','product_id','sku','discount','category','brand_name']].drop_duplicates()

In [92]:
ret_1.groupby('retailer_id')['product_id'].count().reset_index()['product_id'].mean()

30.887769398157808

In [93]:
ret_1=ret_1.groupby('retailer_id').head(30)

In [94]:
len(ret_1)

995112

In [95]:
ret_1

,retailer_id,product_id,sku,discount,category,brand_name
0,2062,1767.0,هلو بيبي ماكسي مقاس5- 40 حفاضة,2.0,حفاضات أطفال,هلو بيبي
1,2062,6140.0,سباركل شامبو مشكل + 10 جنيه بضاعه مجانيه - 40...,2.3,شامبو و شاور جيل,سباركل
2,2062,6098.0,حفاضات جود كير مقاس 4 - 100 حفاضة,2.2,حفاضات أطفال,جود كير
3,2062,1766.0,هلو بيبي كبير مقاس 4- 40 حفاضة,2.3,حفاضات أطفال,هلو بيبي
4,2062,6097.0,حفاضات جود كير مقاس 5 - 100 حفاضة,2.4,حفاضات أطفال,جود كير
...,...,...,...,...,...,...
1036150,714672,5153.0,ريد بل ابيض بجوز الهند و التوت - 250 مل,2.5,مشروبات الطاقة,ريد بل
1036152,715825,5153.0,ريد بل ابيض بجوز الهند و التوت - 250 مل,2.5,مشروبات الطاقة,ريد بل
1036155,724250,5153.0,ريد بل ابيض بجوز الهند و التوت - 250 مل,2.5,مشروبات الطاقة,ريد بل
1036157,727899,5153.0,ريد بل ابيض بجوز الهند و التوت - 250 مل,2.5,مشروبات الطاقة,ريد بل


# Specific Skus ***

### ADD SKUs across all WHs

In [ ]:
# upload specific sku list format ==> {product_id,'sku','discount'} mai and magdy

In [ ]:
specific_list1 = pd.read_clipboard()
specific_list1
# specific_list1=specific_list1.rename(columns={'brand':'brand_name'})

In [ ]:
active_retailers['index'] = 1
specific_list1['index'] = 1
# join retailer list to sku list
list_skus1 = pd.merge(active_retailers,specific_list1,on=['index'],how='inner')
# select needed columns
# list_skus1 = list_skus1[['retailer_id','product_id','sku','discount','brand_name']]

In [ ]:
len(list_skus1)

## add specific skus (condition)

In [ ]:
# # retailers current wallet
# bought_cat = ret_metabase(58554)
# bought_cat = change_columns_type(bought_cat)
# bought_cat['buys']=1

In [ ]:
# active_retailers=active_retailers.merge(bought_cat[['retailer_id','buys']],how='left',on='retailer_id')
# active_retailers['buys']=active_retailers['buys'].fillna(0)

In [ ]:
# specific_list3 = pd.read_clipboard()
# len(specific_list3)

In [ ]:
# list_skus3 = active_retailers.merge(specific_list3,how='inner',left_on='buys',right_on='buys')
# len(list_skus3)

### Add SKUs Per WH

In [ ]:
# specific_list2 = pd.read_clipboard()
# specific_list2

In [ ]:
# list_skus2 = active_retailers.merge(specific_list2,how='inner',left_on='warehouse',right_on='wh')
# len(list_skus2)

### Concat

In [ ]:
# list_skus=list_skus1.copy().drop_duplicates(subset = ['product_id','retailer_id'],keep='last')
# list_skus = list_skus[['retailer_id','product_id','sku','discount']]

In [ ]:
# list_skus=pd.concat([list_skus3]).drop_duplicates(subset = ['product_id','retailer_id'],keep='last')
# try:
#     list_skus = list_skus[['retailer_id','wh_id','product_id','brand','sku','discount']]
#     print('brand')
# except:
#     list_skus = list_skus[['retailer_id','wh_id','product_id','sku','discount']]
# len(list_skus)

In [ ]:
# join both dataframes
# frames = [ret_1,list_skus1]
# df = pd.concat(frames)
# df = df.drop_duplicates()
# len(df)

### Remove Rets with Specific SKUs

## Frozen

In [ ]:
frozen_rets = ret_metabase(59445)
frozen_rets = change_columns_type(frozen_rets)

In [ ]:
frozen_rets['flag']=1
yy = list_skus1.merge(frozen_rets[['retailer_id','product_id','flag']], on=['retailer_id','product_id'], how='left')
list_skus1 = yy[yy['flag']!=1]
len(list_skus1)

##### P&G

In [ ]:
# p_g_rets=ret_metabase(56010)
# p_g_rets.columns = p_g_rets.columns.str.lower()
# change_columns_type(p_g_rets)

In [ ]:
# p_g_rets['flag']=1
# yy = df.merge(p_g_rets[['retailer_id','product_id','flag']], on=['retailer_id','product_id'], how='left')
# df = yy[yy['flag']!=1]
# len(df)

##### Rets Bought JBDMB

In [ ]:
# jbdmb_rets=ret_metabase(56856)
# jbdmb_rets.columns = jbdmb_rets.columns.str.lower()
# jbdmb_rets.drop_duplicates(subset = ['retailer_id', 'brand_name'], inplace = True)
# remove_ret_sku=change_columns_type(jbdmb_rets)
# remove_ret_sku.rename(columns={'retailer_id':'retailer_id_rem'},inplace=True)

In [ ]:
# remove_ret_sku['retailer_id_rem']=remove_ret_sku['retailer_id_rem'].astype('string')
# list_skus['retailer_id']=list_skus['retailer_id'].astype('string')

In [ ]:
# df=df.merge(remove_ret_sku,left_on=['retailer_id','brand_name'],right_on=['retailer_id_rem','brand_name'],how='left')
# df=df[df['retailer_id_rem'].isna()]
# df = df.drop(columns=['retailer_id_rem'])
# len(df)

# Join Data frames and packing units **CHECK FIRST CELL

In [ ]:
# join both dataframes
frames = [ret_1,list_skus1]
df = pd.concat(frames)

In [ ]:

df = df[['retailer_id','product_id','category','brand_name','sku','discount']]
df = df.drop_duplicates()
df=df.drop_duplicates(subset = ['product_id','retailer_id'],keep='last')
len(df)

In [ ]:
#find rows with product_id is null
df[df['product_id'].isna()]


In [ ]:
#count the number of product_id per retailer_id
df.groupby('retailer_id')['product_id'].transform('count')

In [ ]:
df.head()

In [ ]:
#add_skus = pd.read_clipboard()

In [ ]:
#df = pd.concat([df,add_skus])

In [ ]:
# merge packing unit to original data frame
sku_pu = pd.merge(df,packing_units, on=['product_id'],how='left')


In [ ]:
# select needed columns
skus_warehouse_stock = sku_pu[['retailer_id','product_id','category','brand_name','sku_x','discount','packing_unit_id']].copy()
# skus_warehouse_stock = skus_warehouse_stock.dropna()

In [ ]:
# drop duplicates
skus_warehouse_stock =skus_warehouse_stock.sort_values(by=['retailer_id','product_id','packing_unit_id','discount'], ascending=[False,False,False,False])
skus_warehouse_stock = skus_warehouse_stock.drop_duplicates(subset=['retailer_id','product_id','packing_unit_id'], keep='first')


In [ ]:
skus_warehouse_stock['retailer_id'].nunique()

In [ ]:
df.to_csv(dirr+'/ret-sku_list.csv',index=False)

# drop inactive retailers and inactive products

In [ ]:
# read not active retailers
active = ret_metabase(22915)

In [ ]:
active

In [ ]:
change_columns_type(active)

In [ ]:
#skus_warehouse_stock = skus_warehouse_stock[~skus_warehouse_stock['retailer_id'].isin(electronics['retailer_id'])]
skus_warehouse_stock['retailer_id'] = skus_warehouse_stock['retailer_id'].apply(int)
skus_warehouse_stock = skus_warehouse_stock[skus_warehouse_stock['retailer_id'].isin(active['retailer_id'])]

In [ ]:
# drop duplicates values
skus_warehouse_stock = skus_warehouse_stock.drop_duplicates()
skus_warehouse_stock['retailer_id'].nunique()

In [ ]:
skus_warehouse_stock=skus_warehouse_stock.rename(columns={'sku_x':'sku'})

In [ ]:
print(len(skus_warehouse_stock))
# drop any duolicated data
skus_warehouse_stock = skus_warehouse_stock[['retailer_id','product_id','sku','category','brand_name','discount','packing_unit_id']].drop_duplicates()
skus_warehouse_stock['retailer_id'] = skus_warehouse_stock['retailer_id'].apply(int)
skus_warehouse_stock['product_id'] = skus_warehouse_stock['product_id'].apply(int)
skus_warehouse_stock =skus_warehouse_stock.sort_values(by=['retailer_id','product_id','packing_unit_id','discount'], ascending=[False,False,False,False])
skus_warehouse_stock = skus_warehouse_stock.drop_duplicates(subset=['retailer_id','product_id','packing_unit_id'], keep='first')
print(len(skus_warehouse_stock))

In [ ]:
# active products
active_prod = ret_metabase(28232)

In [ ]:
change_columns_type(active_prod)

In [ ]:
# drop inactive products
skus_warehouse_stock = skus_warehouse_stock[skus_warehouse_stock['product_id'].isin(active_prod['product_id'])]

In [ ]:
skus_warehouse_stock

In [ ]:
dist_ret_sku=skus_warehouse_stock.copy()

# Upload format

In [ ]:
#round the discount to 2 decimals
skus_warehouse_stock['discount'] = skus_warehouse_stock['discount'].round(3)


In [ ]:
skus_warehouse_stock = skus_warehouse_stock.merge(ret_warehouse,on='retailer_id')

In [ ]:
#skus_warehouse_stock=skus_warehouse_stock[(skus_warehouse_stock['category']!='حاجه ساقعه' )]

In [ ]:
skus_warehouse_stock=skus_warehouse_stock[['retailer_id','product_id','sku','category','brand_name','discount','packing_unit_id']].drop_duplicates()

## Price ups

In [ ]:
price_up = ret_metabase(57178)
price_up = change_columns_type(price_up)

In [ ]:
price_up_final = price_up[['sku_id','brand','sku','price_up_date','new_pp','wac1']]
price_up_final['price_up_date'] = pd.to_datetime(price_up_final['price_up_date'])
price_up_final['days'] = (price_up_final['price_up_date'] - datetime.now()).dt.days
price_up_final['diff_bet_wac'] = price_up_final['new_pp'] - price_up_final['wac1']

In [ ]:
conditions = [
    (price_up_final['days'].between(-1, 3)),
    (price_up_final['days'].between(-3, -2) & (price_up_final['diff_bet_wac'] > 0)),
    (price_up_final['days'].between(-3, -2) & (price_up_final['diff_bet_wac'] < 0))
]

choices = [
    'within 3 days',
    '2-3 days ago - remove',
    '2-3 days ago - push'
]

price_up_final['status'] = np.select(conditions, choices, default='good')
price_up_final = price_up_final[price_up_final['status'].isin(['within 3 days','2-3 days ago - remove'])]

In [ ]:
skus_warehouse_stock = skus_warehouse_stock[~skus_warehouse_stock['product_id'].isin(price_up_final['sku_id'])]

## ------------------------------------------------------------------------------------

In [ ]:
# #set rows with product_id = 1176 and 1177 to discount = 1.5             @mai and magdy edit here
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==305,'discount'] =0.5  # beity
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==307,'discount'] =0.5  # beity
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==143,'discount'] =0.5  # beity
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==144,'discount'] =0.5  # beity
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==145,'discount'] =0.5  # beity
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==146,'discount'] =0.5  # beity
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==11507,'discount'] =0.5  # beity
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==3,'discount'] =1  # haboba
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==23,'discount'] =1  # haboba
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==1776,'discount'] =1  # best
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==1777,'discount'] =1  # best
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==5151,'discount'] = 0.5 # redbull
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==7885,'discount'] = 1 # 3arosa
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==7886,'discount'] = 1 # 3arosa
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==10825,'discount'] = 1 # 3arosa

skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==130,'discount'] = 0.5 #  bekhero 0.5
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==151,'discount'] = 0.5 # bekhero 1L

skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==5165,'discount'] = 1 # sofy
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==9507,'discount'] = 1 # heinz 16 gm
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==10529,'discount'] = 1 # bebem
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==12979,'discount'] = 1 # always
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==159,'discount'] = 1 # knor
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==6098,'discount'] = 1 
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==6097,'discount'] = 1 
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==83,'discount'] = 0.75 #obourland
skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==439,'discount'] = 1 

skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==2424,'discount'] = 0.75




# skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==13028,'discount'] = 0.5 #larsh
# skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==10825,'discount'] = 1 #haircode
# skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==159,'discount'] = 1 #fine foods
# skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==12979,'discount'] = 1 #always maxi
# skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==161,'discount'] = 1 #maggi
# skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==11671,'discount'] = 1 #oxi
# skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==10529,'discount'] = 1 # bi bim 
# skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==9507,'discount'] = 1 # heinz
# skus_warehouse_stock.loc[skus_warehouse_stock['product_id']==12224,'discount'] = 1 # indomie

# #Remove SKUs
# skus_warehouse_stock = skus_warehouse_stock[skus_warehouse_stock['product_id']!=248 ]# juhayna
# skus_warehouse_stock = skus_warehouse_stock[skus_warehouse_stock['product_id']!=409]# juhayna
skus_warehouse_stock = skus_warehouse_stock[skus_warehouse_stock['product_id']!=11827] # scratch cards
skus_warehouse_stock = skus_warehouse_stock[skus_warehouse_stock['product_id']!=11922] # scratch cards
skus_warehouse_stock = skus_warehouse_stock[skus_warehouse_stock['product_id']!=11923] # scratch cards
#skus_warehouse_stock = skus_warehouse_stock[skus_warehouse_stock['product_id']!=5151] # redbull
skus_warehouse_stock = skus_warehouse_stock[skus_warehouse_stock['product_id']!=431] # shahd sokkar
skus_warehouse_stock = skus_warehouse_stock[skus_warehouse_stock['product_id']!=12979] #always


skus_warehouse_stock = skus_warehouse_stock[skus_warehouse_stock['product_id']!=7885] #3arosa
skus_warehouse_stock = skus_warehouse_stock[skus_warehouse_stock['product_id']!=7886] #3arosa
skus_warehouse_stock = skus_warehouse_stock[skus_warehouse_stock['product_id']!=10825] #3arosa
skus_warehouse_stock = skus_warehouse_stock[skus_warehouse_stock['product_id']!=326] #pepsi star


# skus_warehouse_stock = skus_warehouse_stock[skus_warehouse_stock['product_id']!=13028] # larsh



# #Brands
skus_warehouse_stock.loc[skus_warehouse_stock['brand_name']=='سوفي','discount']= 1
# skus_warehouse_stock.loc[skus_warehouse_stock['brand_name'] == 'تانج', 'discount']=0.75
skus_warehouse_stock.loc[skus_warehouse_stock['brand_name']=='فيوري','discount']=0.5

skus_warehouse_stock.loc[skus_warehouse_stock['brand_name']=='اندومي','discount']=1

skus_warehouse_stock.loc[skus_warehouse_stock['brand_name']=='بيبسي','discount']=1



# skus_warehouse_stock = skus_warehouse_stock[skus_warehouse_stock['brand_name']!= 'اندومي']
skus_warehouse_stock = skus_warehouse_stock[skus_warehouse_stock['brand_name']!= 'فيوري']





In [ ]:
#set packing_unit_id to int
skus_warehouse_stock['packing_unit_id'] = skus_warehouse_stock['packing_unit_id'].apply(int)

In [ ]:
#join skus_warehouse_stock with maxab_price on product_id
skus_warehouse_stock = pd.merge(skus_warehouse_stock, maxab_price, on='product_id', how='left')

#join skus_warehouse_stock with sku_constraints on product_id keeping only min_price
skus_warehouse_stock = pd.merge(skus_warehouse_stock, sku_constraints[['product_id','min_price']], on='product_id', how='left')

#remove rows with maxab_price = min_price
skus_warehouse_stock = skus_warehouse_stock[skus_warehouse_stock['maxab_price']!=skus_warehouse_stock['min_price']]

# add new column 'price after discount' = maxab_price * (100 - discount)%
skus_warehouse_stock['price_after_discount'] = skus_warehouse_stock['maxab_price']*(1-skus_warehouse_stock['discount']/100)

#add new coulmn 'discount with constraints' = (maxab_price - min_price) / maxab_price * 100
skus_warehouse_stock['discount_with_constraints'] = ((skus_warehouse_stock['maxab_price']-skus_warehouse_stock['min_price'])/skus_warehouse_stock['maxab_price'])*100

# in a new df keep only rows where price_after_discount < min_price then set discount
modif_skus = skus_warehouse_stock[skus_warehouse_stock['price_after_discount']<skus_warehouse_stock['min_price']]

# remove skus with discount with constraints < 0.5
excluded_skus =skus_warehouse_stock[skus_warehouse_stock['discount_with_constraints']<0.5]
skus_warehouse_stock = skus_warehouse_stock[~skus_warehouse_stock['product_id'].isin(excluded_skus['product_id'])]
# #if price_after_discount < min_price then set discount =  discount with constraints else keep discount
skus_warehouse_stock['discount'] = np.where(skus_warehouse_stock['price_after_discount']<skus_warehouse_stock['min_price'],skus_warehouse_stock['discount_with_constraints'],skus_warehouse_stock['discount'])                                  

In [ ]:
skus_warehouse_stock['discount'] = skus_warehouse_stock['discount'].round(2)

In [ ]:
#check for unique porudct_id
skus_warehouse_stock['product_id'].nunique()

In [ ]:
#count unique products per retailer_id and sort desc
skus_warehouse_stock['retailer_id'].value_counts().sort_values(ascending=False)

In [ ]:
retailer_counts = skus_warehouse_stock['retailer_id'].value_counts()
valid_retailers = retailer_counts[retailer_counts > 10].index
skus_warehouse_stock = skus_warehouse_stock[skus_warehouse_stock['retailer_id'].isin(valid_retailers)]

In [ ]:
# round discount to 3 decimals
skus_warehouse_stock['discount'] = skus_warehouse_stock['discount'].round(3)
skus_warehouse_stock

In [ ]:
#drop rowss with null discount
skus_warehouse_stock = skus_warehouse_stock[skus_warehouse_stock['discount'].notnull()]

In [ ]:
count_check=skus_warehouse_stock.copy()
count_skus = count_check.groupby(['product_id','sku_x']).agg({'retailer_id':'count','discount':'mean'}).reset_index()

In [ ]:
count_skus.to_excel(dirr+'/count_skus.xlsx',index=False)

In [ ]:
# drop nan values
final_2 = skus_warehouse_stock
# concat product id, packing unit id and discount in one column
final_2['Discounts'] = '['+final_2['product_id'].astype(str)+','+final_2['packing_unit_id'].astype(str)+','+final_2['discount'].astype(str)+']'

In [ ]:
skus_warehouse_stock['product_id'].nunique()

In [ ]:
SKU_discount_grouped = skus_warehouse_stock
SKU_discount_grouped['HH_data'] = '['+(SKU_discount_grouped['product_id']).astype(str)+','+(SKU_discount_grouped['packing_unit_id']).astype(str)+','+(SKU_discount_grouped['discount']).astype(str)+']'

In [ ]:
SKU_discount_grouped['product_id'].nunique()

In [ ]:
# set a list of all retailers products
SKU_discount_grouped  = SKU_discount_grouped.groupby('retailer_id')['HH_data'].apply(list).reset_index()
SKU_discount_grouped['Discounts']= SKU_discount_grouped['HH_data'].astype(str).str.replace("'",'').str.replace(' ','')
# group common retailers for each group skus
SKU_discount_grouped = SKU_discount_grouped.groupby('Discounts')['retailer_id'].agg(list).reset_index()

In [ ]:
final_2 = SKU_discount_grouped.copy()

In [ ]:
final_2['Arabic Offer Name']= 'خصوماتك الخاصة'

# Add Date ***

In [ ]:
# Add Happy Hours Start and End time
final_2['Start Date/Time'] = '27/05/2025 12:10'
final_2['End Date/Time'] = '27/05/2025 13:59'

In [ ]:
final_2 = final_2[['retailer_id','Start Date/Time','End Date/Time','Discounts','Arabic Offer Name']]
final_2['French Offer Name']=np.nan
final_2['English Offer Name']=np.nan

In [ ]:
final_2

In [ ]:
# set retailers to 100 per row
data = []
for i,row in final_2.iterrows():
    
    start_date = row['Start Date/Time']
    end_date = row['End Date/Time']
    retailers = row['retailer_id']
    discount = row['Discounts']
    name = row['Arabic Offer Name'] 
    name_f = row['French Offer Name'] 
    name_e = row['English Offer Name'] 
    
    length = len(retailers)
    if(length>100):
        iters = length//100
        remaining = length%100
        for j in range(0,iters+1):
            if(j<=iters):
                start = (j*100)
                end = (j+1)*100
                rets = retailers[start:end]
                data.append({'Discounts':discount,'retailer_id':rets,'Start Date/Time':start_date,'End Date/Time':end_date
                            ,'Arabic Offer Name':name,'French Offer Name':name_f,'English Offer Name':name_e})
            else:
                print("else new")
            
    else:
        data.append({'Discounts':discount,'retailer_id':retailers,'Start Date/Time':start_date,'End Date/Time':end_date
                            ,'Arabic Offer Name':name,'French Offer Name':name_f,'English Offer Name':name_e})
        
dfx = pd.DataFrame(data)
dfx

In [ ]:
dfx.columns

In [ ]:
dfx['English Offer Name'] = 'Special Discounts'
dfx['Swahili Offer Name'] = ''
dfx['Rwandan Offer Name'] = ''

In [ ]:
dfx

In [ ]:
#change column name retailer_id to Retailers List
dfx.rename(columns={'retailer_id': 'Retailers List'}, inplace=True)

In [ ]:
dfx = dfx[['Retailers List','Start Date/Time','End Date/Time','Discounts','Arabic Offer Name','French Offer Name','English Offer Name','Swahili Offer Name','Rwandan Offer Name']]
# 500 row per sheet 
final=dfx.reset_index().drop(columns='index')
mino=final.index.min()
maxo=final.index.max()
ran = [i for i in range(mino,maxo,1000)]
for i in tqdm(range(len(ran))):
    if i+1 == len(ran):
        val1 = ran[i]
        val2 = maxo
    else:
        val1 = ran[i]
        val2 = ran[i+1] - 1
    x=final.loc[val1:val2,:]
    x.to_excel(dirr+'/o_happy_hour_{0}.xlsx'.format(i),index=False)

In [ ]:
dfx

# WonderPush

In [ ]:
air1 =skus_warehouse_stock['retailer_id'].drop_duplicates().reset_index()
air1 = air1[['retailer_id']]
air_list=pd.DataFrame()
air_list['retailers']=air1['retailer_id']
air_list['retailers']='EG_retailers_'+air1['retailer_id'].apply(int).apply(str)
air_list['named_user']='named_user'
air_list=air_list[['named_user','retailers']]
air_list.to_csv(dirr+'/HH_airship_ret.csv', header=False,index=False,encoding='utf-8')
